<a href="https://colab.research.google.com/github/polivio/PEE/blob/master/DecisionTree_KDD_BR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# https://www.kaggle.com/c/kddbr-2019/data

# KNN KDD BR -Polivio
import numpy as np
from sklearn import preprocessing, neighbors
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from math import sqrt




In [2]:
# preprocessing # blob # raw
train = pd.read_csv('https://github.com/polivio/PEE/raw/master/Training_grouped.csv')
test = pd.read_csv('https://github.com/polivio/PEE/raw/master/Test_grouped.csv')


train.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 21 columns):
Unnamed: 0                          80000 non-null int64
traininglabelorder.scatterplotID    80000 non-null int64
qtde.ponto                          80000 non-null int64
Negative                            80000 non-null int64
Sample                              80000 non-null int64
mean.signalX                        80000 non-null float64
sd.signalX                          80000 non-null float64
var.signalX                         80000 non-null float64
kurtosis.signalX                    80000 non-null float64
skew.signalX                        80000 non-null float64
mean.signalY                        80000 non-null float64
sd.signalY                          80000 non-null float64
var.signalY                         80000 non-null float64
kurtosis.signalY                    80000 non-null float64
skew.signalY                        80000 non-null float64
L               

In [3]:
#train.isna().sum()
##X                            4592
##XY                          19463
##Y                            3711

# Dica Mateus substituição pela média
train['X'].fillna(train['X'].mean(),inplace=True)
train['XY'].fillna(train['XY'].mean(),inplace=True)
train['Y'].fillna(train['XY'].mean(),inplace=True)

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 21 columns):
Unnamed: 0                          80000 non-null int64
traininglabelorder.scatterplotID    80000 non-null int64
qtde.ponto                          80000 non-null int64
Negative                            80000 non-null int64
Sample                              80000 non-null int64
mean.signalX                        80000 non-null float64
sd.signalX                          80000 non-null float64
var.signalX                         80000 non-null float64
kurtosis.signalX                    80000 non-null float64
skew.signalX                        80000 non-null float64
mean.signalY                        80000 non-null float64
sd.signalY                          80000 non-null float64
var.signalY                         80000 non-null float64
kurtosis.signalY                    80000 non-null float64
skew.signalY                        80000 non-null float64
L               

In [4]:
train = train.loc[:,['qtde.ponto', 
       'mean.signalX', 'sd.signalX', 'kurtosis.signalX',
      'mean.signalY', 'sd.signalY', 
       'kurtosis.signalY', 'L', 'X', 'XY', 'Y',
       'mean.silhouette', 'traininglabelorder.score']]

## Quando iremos usar os dados de teste?
test = test.loc[:,[ 'qtde.ponto', 
       'mean.signalX', 'sd.signalX', 'kurtosis.signalX',
      'mean.signalY', 'sd.signalY', 
       'kurtosis.signalY', 'L', 'X', 'XY', 'Y',
       'mean.silhouette']]

train.head(20)







,qtde.ponto,mean.signalX,sd.signalX,kurtosis.signalX,mean.signalY,sd.signalY,kurtosis.signalY,L,X,XY,Y,mean.silhouette,traininglabelorder.score
0,365,3164.835616,804.757625,3.516731,1586.430137,568.651580,3.170632,4,92.0,269.000000,80.421065,0.840146,0.009647
1,384,1737.776042,944.798845,2.475172,2999.940104,1716.790542,2.195067,4,82.0,49.000000,249.000000,0.964427,0.842084
2,373,1139.621984,866.460225,1.922726,1836.640751,950.478945,1.436889,4,156.0,7.000000,206.000000,0.936210,0.744601
3,384,3609.692708,1808.877599,1.985210,2469.164062,1219.231193,1.913381,4,100.0,190.000000,90.000000,0.896070,0.842084
4,383,1867.203655,938.232021,2.176598,2180.908616,1515.205489,1.155226,4,176.0,2.000000,201.000000,0.952810,0.944177
5,384,449.072917,507.413489,29.834375,2910.351562,358.648637,7.460460,4,12.0,80.421065,368.000000,0.968487,1.000000
6,362,2076.953039,655.688216,2.774567,1170.375691,342.483082,2.222238,7,132.0,117.000000,106.000000,0.913787,0.578706
7,200,830.200000,648.128942,9.725987,3241.095000,693.599022,5.545654,5,3.0,22.000000,170.000000,0.916503,1.000000
8,384,3586.317708,1862.356219,2.630635,1683.195312,1436.342429,6.643115,4,326.0,80.421065,54.000000,0.956756,0.944319
9,154,222.168831,270.887643,23.916200,2146.623377,434.285275,4.516220,2,6.0,2.000000,144.000000,0.949731,0.751477


In [5]:
train.head(20)
# change the dtype to 'float64' 
train = train.astype('float64')
train.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 13 columns):
qtde.ponto                  80000 non-null float64
mean.signalX                80000 non-null float64
sd.signalX                  80000 non-null float64
kurtosis.signalX            80000 non-null float64
mean.signalY                80000 non-null float64
sd.signalY                  80000 non-null float64
kurtosis.signalY            80000 non-null float64
L                           80000 non-null float64
X                           80000 non-null float64
XY                          80000 non-null float64
Y                           80000 non-null float64
mean.silhouette             80000 non-null float64
traininglabelorder.score    80000 non-null float64
dtypes: float64(13)
memory usage: 7.9 MB


In [6]:
train.head()

,qtde.ponto,mean.signalX,sd.signalX,kurtosis.signalX,mean.signalY,sd.signalY,kurtosis.signalY,L,X,XY,Y,mean.silhouette,traininglabelorder.score
0,365.0,3164.835616,804.757625,3.516731,1586.430137,568.651580,3.170632,4.0,92.0,269.0,80.421065,0.840146,0.009647
1,384.0,1737.776042,944.798845,2.475172,2999.940104,1716.790542,2.195067,4.0,82.0,49.0,249.000000,0.964427,0.842084
2,373.0,1139.621984,866.460225,1.922726,1836.640751,950.478945,1.436889,4.0,156.0,7.0,206.000000,0.936210,0.744601
3,384.0,3609.692708,1808.877599,1.985210,2469.164062,1219.231193,1.913381,4.0,100.0,190.0,90.000000,0.896070,0.842084
4,383.0,1867.203655,938.232021,2.176598,2180.908616,1515.205489,1.155226,4.0,176.0,2.0,201.000000,0.952810,0.944177


In [0]:
test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 11 columns):
mean.signalX        20000 non-null float64
sd.signalX          20000 non-null float64
kurtosis.signalX    20000 non-null float64
mean.signalY        20000 non-null float64
sd.signalY          20000 non-null float64
kurtosis.signalY    20000 non-null float64
L                   20000 non-null int64
X                   18857 non-null float64
XY                  15215 non-null float64
Y                   19029 non-null float64
mean.silhouette     20000 non-null float64
dtypes: float64(10), int64(1)
memory usage: 1.7 MB


In [20]:

# Dica Mateus substituição pela média
test['X'].fillna(test['X'].mean(),inplace=True)
test['XY'].fillna(test['XY'].mean(),inplace=True)
test['Y'].fillna(test['XY'].mean(),inplace=True)

test = test.astype('float64')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
qtde.ponto          20000 non-null float64
mean.signalX        20000 non-null float64
sd.signalX          20000 non-null float64
kurtosis.signalX    20000 non-null float64
mean.signalY        20000 non-null float64
sd.signalY          20000 non-null float64
kurtosis.signalY    20000 non-null float64
L                   20000 non-null float64
X                   20000 non-null float64
XY                  20000 non-null float64
Y                   20000 non-null float64
mean.silhouette     20000 non-null float64
dtypes: float64(12)
memory usage: 1.8 MB


In [0]:
## Naão deu certo!!!

#Normalização:  https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

from sklearn.preprocessing import StandardScaler
data = [[0, 0], [0, 0], [1, 1], [1, 1]]
print (data)

scaler = StandardScaler()
print(scaler.fit(data))

print(scaler)

##print(scaler.mean_)

##print(scaler.transform(data))

### Devo e como farei para os meus dados??



[[0, 0], [0, 0], [1, 1], [1, 1]]
StandardScaler(copy=True, with_mean=True, with_std=True)
StandardScaler(copy=True, with_mean=True, with_std=True)


In [0]:
##from sklearn.preprocessing import StandardScaler

##train = scaler.fit(train)

train.info()

test.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 12 columns):
mean.signalX                80000 non-null float64
sd.signalX                  80000 non-null float64
kurtosis.signalX            80000 non-null float64
mean.signalY                80000 non-null float64
sd.signalY                  80000 non-null float64
kurtosis.signalY            80000 non-null float64
L                           80000 non-null float64
X                           80000 non-null float64
XY                          80000 non-null float64
Y                           80000 non-null float64
mean.silhouette             80000 non-null float64
traininglabelorder.score    80000 non-null float64
dtypes: float64(12)
memory usage: 7.3 MB


,mean.signalX,sd.signalX,kurtosis.signalX,mean.signalY,sd.signalY,kurtosis.signalY,L,X,XY,Y,mean.silhouette
0,2929.713542,983.267187,2.204854,2338.335938,880.798587,1.852330,7.0,116.0,167.0,94.0,0.916505
1,1409.484375,525.140811,2.798995,3155.841146,978.277806,2.624796,4.0,92.0,189.0,99.0,0.891220
2,2031.329843,2023.975889,2.531504,2982.876963,996.573219,2.506589,5.0,50.0,73.0,254.0,0.947689
3,2900.078378,1278.838337,1.794436,2463.178378,1937.785232,1.229172,5.0,170.0,10.0,185.0,0.954000
4,2567.598958,1090.155787,2.272346,3239.945312,1195.922708,2.315470,4.0,106.0,182.0,92.0,0.829513


In [8]:
# feature qtde.ponto poderá ser removida??


X = np.array(train.drop(['traininglabelorder.score'], 1))

y = np.array(train['traininglabelorder.score'])


print(y)




[0.00964712 0.84208443 0.74460145 ... 0.8007374  0.00594233 0.7671597 ]


In [0]:
# Dados de treinamento e teste - 80.000 objetos que foram divididos.
# X - features, y - lables
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Dados de valição é  o conjunto de treinamento 20.000 objetos

#X_val, y_val 
X_val = np.array(test)

##y_val = "score" --> no site do kaggle?? Como fazer 


In [10]:
# Regressor

clf = tree.DecisionTreeRegressor(max_depth = None)

clf.fit(X_train , y_train)



DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [0]:
y_pred = clf.predict(X_test)


In [12]:
print(y_pred)

[0.795196   1.         0.79495816 ... 1.         1.         0.04426119]


In [14]:
rmse = sqrt(mean_squared_error(y_pred, y_test))
print (rmse)

0.27730811403356553


In [15]:
# !Se todos os outros modelos estão verificando com rmse é o em comum para todos
accuracy = clf.score(X_test, y_test)

print(accuracy)

0.3288602435718553


In [16]:
# sugestão normalizar os dados no caso para KNN, arvore não precisa? ver o que vai modificar. Para os dados do dataset de testes o score é visto no site... com scatterplotID e o score gerado...
print(cross_val_score(clf, X, y, cv=5))
print(cross_val_score(clf, X, y, cv=5).mean())

#tree.plot_tree(clf.fit(X_train , y_train))

[0.30489699 0.32676664 0.32377391 0.32852509 0.32637068]
0.3252504423501749


In [0]:
# Investigar mais sobre cross_validation e normalização no sklearn... e testar com o conjunto de treinamento no kaggle.

In [22]:
# conversão dos dados de treinamento para array para modelo.

Xval = np.array(test)
print(Xval)



[[3.84000000e+02 2.92971354e+03 9.83267187e+02 ... 1.67000000e+02
  9.40000000e+01 9.16504577e-01]
 [3.84000000e+02 1.40948438e+03 5.25140811e+02 ... 1.89000000e+02
  9.90000000e+01 8.91219883e-01]
 [3.82000000e+02 2.03132984e+03 2.02397589e+03 ... 7.30000000e+01
  2.54000000e+02 9.47688522e-01]
 ...
 [3.82000000e+02 4.13408377e+03 2.40434429e+03 ... 4.50000000e+01
  1.26000000e+02 9.02243658e-01]
 [3.84000000e+02 1.09517448e+03 5.62200124e+02 ... 1.20000000e+01
  1.87000000e+02 9.49857651e-01]
 [3.79000000e+02 3.48129288e+03 1.81792133e+03 ... 2.10000000e+02
  9.00000000e+01 8.95854906e-01]]


In [24]:
# Estes dados associar ao Teste - ScatterPlotId  & este score e confirmar no site...
y_pred = clf.predict(Xval)
print(y_pred)



[0.6323367  0.9443186  0.8417066  ... 1.         0.89138154 0.1760756 ]


AttributeError: ignored